In [1]:
from tqdm import tqdm

def create_example(all_data):
    all_result = []
    for data in tqdm(all_data):
        data_id = data["_id"]
        Question = data["question"]
        answer = data["answer"]
        context = data["context"]
        sent_list = []
        sentences = ""
        for index, j in enumerate(context):
            title = j[0]
            sent_list.append(title.strip()+ ".")
            if sentences == "":
                sentences = title + ". "
            else:
                sentences = sentences + " " + title + ". "
            for sent in j[1]:
                sentences = sentences + sent
                sent_list.append(sent.strip())
            
        result = {}
        result["_id"] = data_id
        result["question"] = Question
        result["document"] = sentences
        result["sent"] = sent_list
        result["output"] = answer

        all_result.append(result)

    return all_result
    

In [2]:
import json
file_path = "../data/origin/hotpot_train_v1.1_re.json"
# file_path = "../data/origin/hotpot_dev.json"
with open(file_path, "r", encoding="utf-8") as file:
    dev_data = json.load(file)

input_data = create_example(dev_data)

100%|██████████| 90447/90447 [00:02<00:00, 32040.75it/s]


In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct")

/home/rbqlsquf2/workspace/XAI_rationale-inference-LLM/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from tqdm import tqdm
all_len = []
all_result = []

for example in tqdm(input_data):
    MAX_LENGTH = 2048
    input_ids, attention_mask, labels = [], [], []
    example["document"] = example["document"].strip()
    # token 된 doc
    token_doc = {"input_ids": [], "attention_mask": []}
    # document 문장 index
    sentence_number = 0
    sentence_position = []
    for i, sent in enumerate(example["sent"]):
        # 0번 문장은 instruction으로 지정할 계획
        sent = sent.strip()
        token_sent = tokenizer(sent + " ", add_special_tokens=False)
        sentence_number += 1  # 1부터 시작
        sentence_position.extend([sentence_number] * len(token_sent["input_ids"]))
        token_doc["input_ids"] += token_sent["input_ids"]
        token_doc["attention_mask"] += token_sent["attention_mask"]
    token_end = tokenizer("<|im_end|>\n", add_special_tokens=False)
    sentence_position.extend([0] * len(token_end))
    token_doc["input_ids"] += token_end["input_ids"]
    token_doc["attention_mask"] += token_end["attention_mask"]
    instruction = tokenizer(
        f"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n**Question:{example['question']}\n**Document:\n",
        add_special_tokens=False,
    )
    response = tokenizer(
        f"<|im_start|>assistant\n**Answer:{example['output'].strip()}<|im_end|>\n", add_special_tokens=False
    )
    
    input_ids = instruction["input_ids"] + token_doc["input_ids"] + response["input_ids"]
    count = len(input_ids)
    if count <= MAX_LENGTH:
        all_result.append(example)
    all_len.append(count)

 20%|██        | 18175/90447 [01:14<05:01, 239.45it/s]

In [5]:
print(len(all_result))
print(sum(all_len)/len(all_len))

90447
86434


In [ ]:
file_path = "../1112data/hotpot_train.json"
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [6]:
with open("../data/1008data/hotpot_train.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [7]:
from tqdm import tqdm
all_len = []
all_result = []
over_num = 0
for input_data_ in tqdm(input_data):
    text = input_data_["all_text"]
    count = len(tokenizer(text)["input_ids"])
    if count <= 2048:
        all_result.append(input_data_)
    else:
        over_len = count - 2048
        input_data_['text'] = input_data_['text'][:over_len]
        over_num +=1
    # all_len.append(len(tokenizer(text)["input_ids"]))
print(over_num)

100%|██████████| 7405/7405 [00:18<00:00, 405.87it/s]

668


In [8]:
for input_data_ in tqdm(input_data):
    del input_data_["all_text"]

100%|██████████| 7405/7405 [00:00<00:00, 1753249.85it/s]


In [10]:
with open("../data/qwen_hotpot_test_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [37]:
from tqdm import tqdm
all_len = []
all_result = []

for input_data_ in tqdm(input_data):
    text = input_data_["text"]
    if len(tokenizer(text)["input_ids"]) <= 2048:
        # data["text"] = data["text"]
        all_result.append(input_data_)
    # all_len.append(len(tokenizer(text)["input_ids"]))

100%|██████████| 90447/90447 [04:12<00:00, 357.70it/s]


In [38]:
with open("data/qwen_hotpot_train_data.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)

In [16]:
with open("data/teddst_dev.json", "w", encoding="utf-8") as f:
    json.dump(input_data, f, ensure_ascii=False, indent=4)

In [7]:
print(input_data[0]["label"])

**Answer**: yes
**Supporting Sentences**: [4] Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.
[17] Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.



In [16]:
count = len(list(filter(lambda x: x < 2048, all_len)))
print(count)

6539
